Importing libraries and the preprocessed data

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math
import warnings
warnings.filterwarnings('ignore')

X_train=pd.read_csv('X_train').drop(labels=['Site Time'], axis=1) #the timestamp data which was the index is dropped
X_test=pd.read_csv('X_test').drop(labels=['Site Time'], axis=1)
y_train=pd.read_csv('y_train').drop(labels=['Site Time'], axis=1)
y_test=pd.read_csv('y_test').drop(labels=['Site Time'], axis=1)

The first run of grid search and cross validation, fitting a support vector machine to the training data

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer
from sklearn.svm import SVR #support vector regression

def MSE(y_true,y_pred): #creating a root mean squared error scorer from the mean squared error
    mse =  math.sqrt(mean_squared_error(y_true, y_pred))
    return mse

tuned_parameters = [{'kernel': ['rbf'], 'gamma': np.logspace(-5,2, num=5),
                     'C': np.logspace(0,4,num=5)},
                    {'kernel': ['linear'], 'C': np.logspace(0,4,num=5)}] #the hyperparameter grid to search
reg = GridSearchCV( #10 for epsilon means not to penalize any predictions with a difference of less 10 from the target
    SVR(epsilon=10), tuned_parameters, n_jobs=-1, refit=True, verbose=5, cv=TimeSeriesSplit(n_splits=4), scoring=make_scorer(MSE, greater_is_better=False)#this search will be run parallel, the final best model refitted using the entire training data, with updates through output, cross validation folds generated through a splitter specific for time series data, and the MSE function will be used for scoring 
)
reg.fit(X_train, y_train)

Fitting 4 folds for each of 30 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  86 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 105 out of 120 | elapsed:   14.3s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   41.5s finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=4),
             estimator=SVR(epsilon=10), n_jobs=-1,
             param_grid=[{'C': array([1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
                          'gamma': array([1.00000000e-05, 5.62341325e-04, 3.16227766e-02, 1.77827941e+00,
       1.00000000e+02]),
                          'kernel': ['rbf']},
                         {'C': array([1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
                          'kernel': ['linear']}],
             scoring=make_scorer(MSE, greater_is_better=False), verbose=5)

Evaluating the accuracy after the first run

In [19]:
print(reg.best_params_) #the set of best parameters found

import joblib

predict=pd.DataFrame(reg.predict(X_test)) #generating the prediction

time_transformer=joblib.load('time_transformer') #retrieving the scaler so that only the daylight values are evaluated

t=predict.join(X_test).join(y_test) #putting all three DataFrames together
t=t.rename(columns={0:'predict'})
t['time']=time_transformer.inverse_transform(pd.DataFrame(t['time'])) #reversing the scaling done to the cyclical time column
t=t[t['time']>0] #keeping only values that are during the day

mod_predict=t['predict'].copy() #extracting the daylight-only predictions
mod_y_test=t['one ahead power Kilowatts'].copy() #extracting the daylight-only target values

print("r2 score: "+str(r2_score(mod_predict, mod_y_test))) #r2
print("Root mean squared error: "+str(math.sqrt(mean_squared_error(mod_predict, mod_y_test)))) #rmse

{'C': 10000.0, 'gamma': 0.03162277660168379, 'kernel': 'rbf'}
r2 score: 0.7344351604680259
Root mean squared error: 139.81742996329479


The second grid search run, a refinement done by focusing on the area near the first search's results

In [4]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': np.logspace(-2, 1, num=5),
                     'C': np.logspace(2,4,num=5)}] #values will change depending on the parameters found by the first run
reg = GridSearchCV(
    SVR(epsilon=10), tuned_parameters, n_jobs=-1, refit=True, verbose=5, cv=TimeSeriesSplit(n_splits=4), scoring=make_scorer(MSE, greater_is_better=False)
)
reg.fit(X_train, y_train)

Fitting 4 folds for each of 25 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 19.0min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=4),
             estimator=SVR(epsilon=10), n_jobs=-1,
             param_grid=[{'C': array([  100.        ,   316.22776602,  1000.        ,  3162.27766017,
       10000.        ]),
                          'gamma': array([ 0.01      ,  0.05623413,  0.31622777,  1.77827941, 10.        ]),
                          'kernel': ['rbf']}],
             scoring=make_scorer(MSE, greater_is_better=False), verbose=5)

Evaluating the accuracy after the second run

In [6]:
print(reg.best_params_)

predict=pd.DataFrame(reg.predict(X_test))

time_transformer=joblib.load('time_transformer')

t=predict.join(X_test).join(y_test)
t=t.rename(columns={0:'predict'})
t['time']=time_transformer.inverse_transform(pd.DataFrame(t['time']))
t=t[t['time']>0]

mod_predict=t['predict'].copy()
mod_y_test=t['one ahead power Kilowatts'].copy()

print("r2 score: "+str(r2_score(mod_predict, mod_y_test)))
print("Root mean squared error: "+str(math.sqrt(mean_squared_error(mod_predict, mod_y_test))))

{'C': 3162.2776601683795, 'gamma': 0.05623413251903491, 'kernel': 'rbf'}
r2 score: 0.9317719449266925
Root mean squared error: 71.99487963876774


Plotting the prediction against the target


In [20]:
import plotly.express as px

predict=predict.rename(columns={0:'Prediction'})
y_test=y_test.rename(columns={'one ahead power Kilowatts': 'Target'})
merged=predict.join(y_test) #joining the target and prediction for plotting

fig = px.line(merged)
fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Power (kw)",
)
fig.show()

Getting the accuracy score for each combination of gamma and C, two hyperparameters

In [5]:
results=pd.DataFrame(reg.cv_results_)[['param_gamma', 'param_C', 'mean_test_score']].dropna() #only keeping the columns for gamma, C, and the score
results['mean_test_score']=results['mean_test_score']*-1 #the scores are negative, so making them positive

d=np.empty(shape=(len(list(results['param_C'].unique())), len(list(results['param_gamma'].unique())))) #an array to hold the scores, the columns are the gammas and the rows the Cs
for i in range(len(list(results['param_C'].unique()))): #going through each C value
    for j in range(len(list(results['param_C'].unique()))): #going through each gamma value
        d[i,j]=results.loc[(results['param_C'] == list(results['param_C'].unique())[i]) & (results['param_gamma'] == list(results['param_gamma'].unique())[j])]['mean_test_score'] #getting the score that corresponds to a specific combination of C and gamma. Accessing is done by merging two boolean masks of the results array of which cells have the selected C and gamma

Plotting the contour graph

In [7]:
import plotly.graph_objects as go

fig = go.Figure(data =
    go.Contour(
        z=[list(i) for i in d], #the colors of the plot will be determined by each row
        x=np.log10(list(results['param_gamma'].unique())), 
        y=np.log10(list(results['param_C'].unique())), 
        contours=dict(
            coloring ='heatmap',
            showlabels = True, 
            labelfont = dict( 
                size = 12,
                color = 'white',
            )
    ), 
    colorscale='mint',
    )
)

fig.update_layout(
    xaxis_title="log10 of gamma",
    yaxis_title="log10 of C",
)
fig.show()